In [4]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as agg
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [5]:
month = pd.Grouper(key="date", freq="m")
idx_cols = ["user_id", month]

In [33]:
dfr = hd.read_raw_data("777", columns=None)
hd.inspect(dfr)

Time for read_raw_data                 : 3.65 seconds
(682,656, 27)


,Transaction Reference,User Reference,User Registration Date,Year of Birth,Salary Range,Postcode,LSOA,MSOA,Derived Gender,Transaction Date,Account Reference,Provider Group Name,Account Type,Latest Recorded Balance,Transaction Description,Credit Debit,Amount,User Precedence Tag Name,Manual Tag Name,Auto Purpose Tag Name,Merchant Name,Merchant Business Line,Account Created Date,Account Last Refreshed,Data Warehouse Date Created,Data Warehouse Date Last Updated,Transaction Updated Flag
0,688293,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,"9572 24jan12 , tcs bowdon , bowdon gb - pos",Debit,25.030001,No Tag,No Tag,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-24,U
1,688291,777,2011-07-20,1969.0,20K to 30K,WA1 4,E01012553,E02002603,M,2012-01-25,262916,NatWest Bank,Current,364.220001,warrington b.c. - d/d,Debit,26.000000,Gym Membership,Gym Membership,No Tag,No Merchant,Unknown Merchant,2011-07-20,2020-07-21 20:32:00,2014-07-18,2017-10-23,U


In [9]:
dft = hd.read_txn_data("777")
hd.inspect(dft)

Time for read_txn_data                 : 3.39 seconds
(649,820, 19)


,date,user_id,amount,desc,merchant,tag_group,tag,account_id,account_last_refreshed,account_provider,account_type,debit,desc_orig,female,id,latest_balance,postcode,tag_auto,yob
0,2012-02-01,777,400.00,mdbremoved,NaN,transfers,other_transfers,262916,2020-07-21 20:32:00,natwest bank,current,True,<mdbremoved> - s/o,0.0,688300,364.220001,wa1 4,transfers,1969.0
1,2012-02-01,777,3.03,aviva pa,aviva,spend,finance,262916,2020-07-21 20:32:00,natwest bank,current,True,aviva pa - d/d,0.0,688299,364.220001,wa1 4,health insurance,1969.0


In [10]:
dft.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649820 entries, 0 to 649819
Data columns (total 19 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    649820 non-null  datetime64[ns]
 1   user_id                 649820 non-null  int32         
 2   amount                  649820 non-null  float32       
 3   desc                    649820 non-null  category      
 4   merchant                323417 non-null  category      
 5   tag_group               508781 non-null  category      
 6   tag                     512365 non-null  category      
 7   account_id              649820 non-null  int32         
 8   account_last_refreshed  649820 non-null  datetime64[ns]
 9   account_provider        649820 non-null  category      
 10  account_type            649820 non-null  category      
 11  debit                   649820 non-null  bool          
 12  desc_orig               649820

In [8]:
fs.ls("3di-project-entropy/txn_777.parquet", detail=True)

[{'Key': '3di-project-entropy/txn_777.parquet',
  'LastModified': datetime.datetime(2022, 2, 4, 17, 49, 50, tzinfo=tzutc()),
  'ETag': '"d74984680d71116d8c67b179e0444ef8-4"',
  'Size': 17155767,
  'StorageClass': 'STANDARD',
  'type': 'file',
  'size': 17155767,
  'name': '3di-project-entropy/txn_777.parquet'}]

In [6]:
dfa = hd.read_analysis_data("777")
hd.inspect(dfa)

Time for read_analysis_data            : 0.91 seconds
(729, 28)


active_accounts  txns_count  \
user_id date                                                               
13777   2012-11-30          [112648, 112650, 112652, 112651]         100   
        2012-12-31  [112648, 112650, 112652, 112651, 112653]          65   

                       txns_value  txn_count_sa  txn_count_ca   month_spend  \
user_id date                                                                  
13777   2012-11-30  317126.781250          14.0          85.0  11073.820312   
        2012-12-31   34068.792969           8.0          52.0   7344.680176   

                    log_month_spend  prop_spend_finance  prop_spend_services  \
user_id date                                                                   
13777   2012-11-30         9.312339            0.103978             0.036452   
        2012-12-31         8.901731            0.001842             0.019319   

                    prop_spend_other_spend  prop_spend_household  \
user_id date                                                       
13777   2012-11-30                0.152251              0.647397   
        2012-12-31                0.140238              0.773029   

                    prop_spend_motor  prop_spend_communication  \
user_id date                                                     
13777   2012-11-30          0.000000                       0.0   
        2012-12-31          0.027594                       0.0   

                    prop_spend_retail  prop_spend_travel  prop_spend_hobbies  \
user_id date                                                                   
13777   2012-11-30           0.045302           0.012362            0.002258   
        2012-12-31           0.025045           0.000000            0.012933   

                    month_income  annual_income  entropy_sptac      region  \
user_id date                                                                 
13777   2012-11-30   6366.030273   69899.824219       2.623084  South East   
        2012-12-31   5283.939941   69899.824219       2.705641  South East   

                     age  female  sa_inflows  sa_outflows  sa_net_inflows  \
user_id date                                                                
13777   2012-11-30  43.0     0.0     50076.0     72909.75       -22833.75   
        2012-12-31  43.0     0.0      3250.0      7837.00        -4587.00   

                    sa_scaled_inflows  sa_scaled_outflows  \
user_id date                                                
13777   2012-11-30           8.596760           12.516727   
        2012-12-31           0.557941            1.345411   

                    sa_scaled_net_inflows  
user_id date                               
13777   2012-11-30              -3.919967  
        2012-12-31              -0.787470

In [8]:
dft

active_accounts             0
txns_count                  0
txns_value                  0
txn_count_sa                0
txn_count_ca                0
month_spend                 0
log_month_spend             0
prop_spend_finance          0
prop_spend_services         0
prop_spend_other_spend      0
prop_spend_household        0
prop_spend_motor            0
prop_spend_communication    0
prop_spend_retail           0
prop_spend_travel           0
prop_spend_hobbies          0
month_income                0
annual_income               0
entropy_sptac               0
region                      0
age                         0
is_female                      0
sa_inflows                  0
sa_outflows                 0
sa_net_inflows              0
sa_scaled_inflows           0
sa_scaled_outflows          0
sa_scaled_net_inflows       0
dtype: int64